In [1]:
from src.system_env import System
from src.RL_brain import DeepQNetwork, QLearningTable
from src.random_walk_brain import RandomWalk
from src.supervised_brain import QLearningSupervisedNeuralNetwork

import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

In [2]:
def update(iteration_max=100, links_clf=None):
    dtimes = []
    for episode in range(iteration_max):
        sys.stdout.flush()
        # initial observation
        observation = env.reset()
        ite_cnt = 0
        while True:
            ite_cnt += 1
            # fresh env
            env.render()
            # RL choose action based on observation
            if not links_clf:
                action = RL.choose_action(observation)
            else:
                action = RL.choose_action(observation,
                                          links_clf.predict_proba(observation.reshape(-1,int(env.n_features/env.n_actions)))[:,1])

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)
            # RL learn from this transition
            RL.store_transition(observation, action, reward, observation_)
            if hasattr(RL, 'memory_counter'):
                if ite_cnt % 5 == 0 and RL.memory_counter > 200:
                    RL.learn()
            else:
                RL.learn(observation, action, reward, done, observation_) # for QLearningTable
            if episode > 0:
                print("\rStep %5d @ Episode %3d/%3d (%5d/%5d)" % (ite_cnt, episode + 1, iteration_max, 
                                                                 dtimes[-1], min(dtimes)), end="")    
            else:
                print("\rStep %d @ Episode %d/%d" % (ite_cnt, episode + 1, iteration_max), end="")    
            # swap observation
            observation = observation_
            # break while loop when end of this episode
            if done:# or ite_cnt > 10000:
                break    
        dtimes.append(ite_cnt)
    # end of game
    print('game over\n')
    env.destroy()
    if hasattr(RL, 'cost_his'):
        return dtimes, RL.cost_his
    return dtimes, []

In [3]:
output_dtimes, output_losses = [], []

In [4]:
env = System(system_h=20, system_w=20, system_fake=0.3)

backbone: [0, 1, 21, 22, 23, 24, 25, 26, 27, 28, 48, 49, 69, 70, 90, 91, 111, 112, 132, 133, 134, 154, 174, 175, 195, 215, 235, 236, 237, 238, 258, 259, 279, 299, 319, 339, 359, 379, 399]
fake node: [3, 5, 6, 7, 8, 9, 13, 15, 19, 20, 33, 37, 39, 51, 52, 53, 61, 72, 75, 76, 79, 81, 86, 87, 94, 96, 97, 98, 101, 105, 106, 108, 109, 113, 119, 120, 124, 125, 136, 138, 140, 142, 143, 148, 150, 161, 168, 183, 185, 188, 189, 196, 197, 203, 204, 205, 210, 211, 212, 214, 217, 218, 223, 227, 234, 243, 245, 248, 252, 257, 260, 261, 264, 265, 268, 269, 270, 273, 274, 276, 278, 283, 285, 287, 288, 289, 293, 297, 301, 302, 304, 305, 307, 309, 311, 314, 320, 325, 326, 328, 329, 333, 338, 342, 343, 346, 347, 354, 358, 362, 363, 365, 366, 368, 370, 373, 374, 388, 389, 391]


In [ ]:
#     X = np.vstack((env.fake_observations, env.normal_observations))
#     y = np.hstack((np.zeros(env.fake_observations.shape[0]), np.ones(env.normal_observations.shape[0])))
#     X, y = shuffle(X, y)
#     clf = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', 
#                         alpha=0.01, batch_size='auto', learning_rate='constant', 
#                         learning_rate_init=0.001, power_t=0.5, max_iter=2000, shuffle=True, 
#                         random_state=None, tol=0.0001, verbose=False, warm_start=False, 
#                         momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
#                         validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#     clf.fit(X, y)

In [5]:
# Q-learning
RL = DeepQNetwork(env.n_actions, env.n_features, reward_decay=0.9, epsilon=0.9, e_greedy_increment=None,
                  replace_target_iter=300, memory_size=10000, batch_size=500)
# RL = QLearningTable(actions=list(range(env.n_actions)), learning_rate=0.5, reward_decay=0.9, epsilon=0.1)

In [ ]:
eposide_max = 5
for i in range(eposide_max):
    print('Epoch {}'.format(i))
    start_clock = time.time()
    dtimes, losses = update(100)
    output_dtimes.append(dtimes)
    output_losses.append(losses)
    end_clock = time.time()
    print('It takes {} s'.format(end_clock - start_clock))

Epoch 0
Step   566 @ Episode 100/100 ( 1166/  167)game over

It takes 1267.4045870304108 s
Epoch 1
Step   577 @ Episode 100/100 (  456/  163)game over

It takes 701.9088673591614 s
Epoch 2
Step  1167 @ Episode 100/100 ( 1311/  158)game over

It takes 687.5665345191956 s
Epoch 3
Step  1257 @ Episode 100/100 ( 1995/  166)game over

It takes 733.1808364391327 s
Epoch 4
Step 25897 @ Episode  73/100 (  264/  200)

In [ ]:
start_clock = time.time()
dtimes, losses = update(100)
output_dtimes.append(dtimes)
output_losses.append(losses)
end_clock = time.time()
print('It takes {} s'.format(end_clock - start_clock))

In [ ]:
RL.reset()

In [ ]:
df = pd.DataFrame.from_dict(RL.q_table, orient='index')
print(df.shape)
print(df.max())
print(df.sum(axis=0))
df

In [ ]:
result_dtimes = np.array(output_dtimes).T
# pd.DataFrame(result_dtimes, columns=['experiment_9']) \
#   .to_csv('results/DQN25_0.3_10_500_0.5_0.1_exp9(28min).csv',index=False)

In [ ]:
plt.figure(figsize=(12, 7))
result_dtimes = np.array(output_dtimes).T
num = 20
plt.subplot(211)
pdt = []
for i,dt in enumerate(result_dtimes.T):
    pdt.extend(list(dt))
#     plt.semilogy([np.mean(dt[i:i+num]) for i in range(len(dt)-num)], label='exp {}'.format(i))
plt.semilogy([np.mean(pdt[i:i+num]) for i in range(len(pdt)-num)], label='exp')
plt.xlabel('epoch')
plt.ylabel('iterations')
plt.legend(ncol=2)
# plt.savefig('figures/DQN25_0.3_10_500_0.5_0.1_exp9(28min).png', dpi=300)
# plt.show()
result_losses = output_losses[-1:]
num = 100
plt.subplot(212)
pss = []
for i,losses in enumerate(result_losses):
    pss.extend(list(losses))
#     plt.semilogy(losses, '-')
plt.semilogy(pss)#([np.mean(pss[i:i+num]) for i in range(len(pss)-num)])
plt.xlabel('epoch')
plt.ylabel('loss')
# plt.savefig('figures/_1000+.png', dpi=300)
plt.show()